### Import libraries

In [90]:
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
import matplotlib as plt
import researchpy as rp
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
import patsy


### Data import and wrangling for model

Let's start with loading the cleaned data after the EDA1

In [4]:
df=pd.read_csv("./houses_data_clean.csv")
df.head()

,Unnamed: 0,Id,MSSubClass,MSZoning,Neighborhood,HouseStyle,OverallQual,YearBuilt,YearRemodAdd,Exterior1st,...,Fireplaces,GarageType,GarageFinish,GarageCars,GarageArea,SalePrice,OverallCond_bin,FullBath_bin,ExterQual_bin,KitchenQual_bin
0,0,142,20,RL,CollgCr,1Story,7,2005,2005,VinylSd,...,0,Attchd,Fin,2,660,260000,1,1,0,0
1,1,170,20,RL,Timber,1Story,8,1981,1981,Plywood,...,1,Attchd,RFn,2,511,228000,0,1,0,1
2,2,303,20,RL,CollgCr,1Story,7,2001,2002,VinylSd,...,1,Attchd,RFn,3,843,205000,1,1,0,0
3,3,371,60,RL,Gilbert,2Story,6,2000,2000,VinylSd,...,1,Attchd,RFn,2,460,172400,1,1,1,1
4,4,412,190,RL,Gilbert,1Story,5,1955,1955,Wd Sdng,...,0,Attchd,Fin,2,572,145000,1,0,1,1


In [5]:
#check column types
pd.value_counts(df.dtypes)

int64     16
object    11
dtype: int64

Object types are most likely string data. Let's check it:

In [6]:
#Inspect which columns are of object type
pd.set_option('display.max_columns', 30)
print(df.select_dtypes(exclude="number"))

     MSZoning Neighborhood HouseStyle Exterior1st Exterior2nd MasVnrType  \
0          RL      CollgCr     1Story     VinylSd     VinylSd       None   
1          RL       Timber     1Story     Plywood     Plywood    BrkFace   
2          RL      CollgCr     1Story     VinylSd     VinylSd    BrkFace   
3          RL      Gilbert     2Story     VinylSd     VinylSd       None   
4          RL      Gilbert     1Story     Wd Sdng     Wd Sdng       None   
...       ...          ...        ...         ...         ...        ...   
1455       RL        NAmes     1Story     VinylSd     VinylSd       None   
1456       RM       IDOTRR     2Story     VinylSd     Wd Shng       None   
1457       RL      Edwards     1.5Fin     MetalSd     MetalSd       None   
1458       RL      NPkVill     2Story     Plywood     Brk Cmn       None   
1459       RL      NPkVill     1Story     Plywood     Plywood       None   

     Foundation BsmtQual BsmtFinType1 GarageType GarageFinish  
0         PConc       G

In [7]:
#Switching data types from object to string
for col in df.columns:
    if df[col].dtype == "object":
        df[col] = pd.Series(df[col], dtype="string")
        
pd.value_counts(df.dtypes)

int64     16
string    11
dtype: int64

The string data will have to be encoded, but so will some of the others, the ones that are more categorical/ordinal. Let's mark those variables, wchich are such

In [8]:
col_ordinal=[col for col in df.select_dtypes(include=["number"]).columns if 1.*df[col].nunique()/df[col].count() < 0.05]
col_ordinal

['MSSubClass',
 'OverallQual',
 'YearRemodAdd',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageCars',
 'OverallCond_bin',
 'FullBath_bin',
 'ExterQual_bin',
 'KitchenQual_bin']

The "$_bin" variables are in fact already encoded, so we should leave them out when encoding in future

In [12]:
col_to_encode=[col for col in df.select_dtypes(exclude=["number"]).columns]
col_to_encode.extend([col for col in col_ordinal if col.find("_bin") == -1])
col_to_encode

['MSZoning',
 'Neighborhood',
 'HouseStyle',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'Foundation',
 'BsmtQual',
 'BsmtFinType1',
 'GarageType',
 'GarageFinish',
 'MSSubClass',
 'OverallQual',
 'YearRemodAdd',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageCars']

So now let's encode using one hot encoding in Pandas

In [27]:
df_encoded=pd.get_dummies(df, columns=col_to_encode, drop_first = True)
df_encoded.head()

,Unnamed: 0,Id,YearBuilt,GrLivArea,GarageArea,SalePrice,OverallCond_bin,FullBath_bin,ExterQual_bin,KitchenQual_bin,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Neighborhood_Blueste,...,TotRmsAbvGrd_6,TotRmsAbvGrd_7,TotRmsAbvGrd_8,TotRmsAbvGrd_9,TotRmsAbvGrd_10,TotRmsAbvGrd_11,TotRmsAbvGrd_12,TotRmsAbvGrd_14,Fireplaces_1,Fireplaces_2,Fireplaces_3,GarageCars_1,GarageCars_2,GarageCars_3,GarageCars_4
0,0,142,2005,1734,660,260000,1,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1,170,1981,1707,511,228000,0,1,0,1,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0
2,2,303,2001,1541,843,205000,1,1,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,3,371,2000,1664,460,172400,1,1,1,1,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0
4,4,412,1955,1056,572,145000,1,0,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


Let's double check if the not encoded columns match?

In [25]:
df[df.columns[~df.columns.isin(col_to_encode)]]

,Unnamed: 0,Id,YearBuilt,GrLivArea,GarageArea,SalePrice,OverallCond_bin,FullBath_bin,ExterQual_bin,KitchenQual_bin
0,0,142,2005,1734,660,260000,1,1,0,0
1,1,170,1981,1707,511,228000,0,1,0,1
2,2,303,2001,1541,843,205000,1,1,0,0
3,3,371,2000,1664,460,172400,1,1,1,1
4,4,412,1955,1056,572,145000,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...
1455,1455,420,1968,1056,304,142000,0,0,1,1
1456,1456,706,1930,1092,0,55000,1,1,1,1
1457,1457,1145,1941,924,280,80000,0,0,1,1
1458,1458,1161,1978,1456,440,146000,1,1,1,1


Luckily for us - it does match :] Yay!
Now, let's look how many entries did we got?

In [30]:
df_encoded.shape

(1460, 198)

### Variables description and assumption check

Before we run the actual model let's look in the whole data how our assumptions are met (i.e. if we have multicolinearity, the probably we should leave out some of the variables). We'll do it on not encoded data

In [31]:
df.columns

Index(['Unnamed: 0', 'Id', 'MSSubClass', 'MSZoning', 'Neighborhood',
       'HouseStyle', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'Foundation', 'BsmtQual', 'BsmtFinType1',
       'GrLivArea', 'TotRmsAbvGrd', 'Fireplaces', 'GarageType', 'GarageFinish',
       'GarageCars', 'GarageArea', 'SalePrice', 'OverallCond_bin',
       'FullBath_bin', 'ExterQual_bin', 'KitchenQual_bin'],
      dtype='object')

In [32]:
df_general_check = df.drop(columns=['Unnamed: 0', 'Id'])
df_general_check.head()

,MSSubClass,MSZoning,Neighborhood,HouseStyle,OverallQual,YearBuilt,YearRemodAdd,Exterior1st,Exterior2nd,MasVnrType,Foundation,BsmtQual,BsmtFinType1,GrLivArea,TotRmsAbvGrd,Fireplaces,GarageType,GarageFinish,GarageCars,GarageArea,SalePrice,OverallCond_bin,FullBath_bin,ExterQual_bin,KitchenQual_bin
0,20,RL,CollgCr,1Story,7,2005,2005,VinylSd,VinylSd,None,PConc,Gd,GLQ,1734,7,0,Attchd,Fin,2,660,260000,1,1,0,0
1,20,RL,Timber,1Story,8,1981,1981,Plywood,Plywood,BrkFace,CBlock,Gd,Unf,1707,6,1,Attchd,RFn,2,511,228000,0,1,0,1
2,20,RL,CollgCr,1Story,7,2001,2002,VinylSd,VinylSd,BrkFace,PConc,Gd,Unf,1541,6,1,Attchd,RFn,3,843,205000,1,1,0,0
3,60,RL,Gilbert,2Story,6,2000,2000,VinylSd,VinylSd,None,PConc,Gd,Unf,1664,7,1,Attchd,RFn,2,460,172400,1,1,1,1
4,190,RL,Gilbert,1Story,5,1955,1955,Wd Sdng,Wd Sdng,None,CBlock,TA,Rec,1056,5,0,Attchd,Fin,2,572,145000,1,0,1,1


Let's encode all strings as integers

In [35]:
for col in df_general_check.select_dtypes(exclude=["number"]).columns:
    df_general_check[col]=df_general_check[col].astype('category')
    name = col+"_cat"
    df_general_check[name] = df_general_check[col].cat.codes
    df_general_check[col]=df_general_check[col].astype('string')
#    print(col)

df_general_check.head()

,MSSubClass,MSZoning,Neighborhood,HouseStyle,OverallQual,YearBuilt,YearRemodAdd,Exterior1st,Exterior2nd,MasVnrType,Foundation,BsmtQual,BsmtFinType1,GrLivArea,TotRmsAbvGrd,...,OverallCond_bin,FullBath_bin,ExterQual_bin,KitchenQual_bin,MSZoning_cat,Neighborhood_cat,HouseStyle_cat,Exterior1st_cat,Exterior2nd_cat,MasVnrType_cat,Foundation_cat,BsmtQual_cat,BsmtFinType1_cat,GarageType_cat,GarageFinish_cat
0,20,RL,CollgCr,1Story,7,2005,2005,VinylSd,VinylSd,None,PConc,Gd,GLQ,1734,7,...,1,1,0,0,3,5,2,12,13,2,2,2,2,1,0
1,20,RL,Timber,1Story,8,1981,1981,Plywood,Plywood,BrkFace,CBlock,Gd,Unf,1707,6,...,0,1,0,1,3,23,2,9,10,1,1,2,5,1,1
2,20,RL,CollgCr,1Story,7,2001,2002,VinylSd,VinylSd,BrkFace,PConc,Gd,Unf,1541,6,...,1,1,0,0,3,5,2,12,13,1,2,2,5,1,1
3,60,RL,Gilbert,2Story,6,2000,2000,VinylSd,VinylSd,None,PConc,Gd,Unf,1664,7,...,1,1,1,1,3,8,5,12,13,2,2,2,5,1,1
4,190,RL,Gilbert,1Story,5,1955,1955,Wd Sdng,Wd Sdng,None,CBlock,TA,Rec,1056,5,...,1,0,1,1,3,8,2,13,14,2,1,3,4,1,0


A quick check if the types are ok?

In [37]:
df_general_check.dtypes

MSSubClass           int64
MSZoning            string
Neighborhood        string
HouseStyle          string
OverallQual          int64
YearBuilt            int64
YearRemodAdd         int64
Exterior1st         string
Exterior2nd         string
MasVnrType          string
Foundation          string
BsmtQual            string
BsmtFinType1        string
GrLivArea            int64
TotRmsAbvGrd         int64
Fireplaces           int64
GarageType          string
GarageFinish        string
GarageCars           int64
GarageArea           int64
SalePrice            int64
OverallCond_bin      int64
FullBath_bin         int64
ExterQual_bin        int64
KitchenQual_bin      int64
MSZoning_cat          int8
Neighborhood_cat      int8
HouseStyle_cat        int8
Exterior1st_cat       int8
Exterior2nd_cat       int8
MasVnrType_cat        int8
Foundation_cat        int8
BsmtQual_cat          int8
BsmtFinType1_cat      int8
GarageType_cat        int8
GarageFinish_cat      int8
dtype: object

We can see that our binarized data are of int64 type. It will be easier for filtering, when they will be the same type as other categorical variables. Let's change that

In [39]:
for col in df_general_check.columns:
    if col.find("_bin") != -1:
        df_general_check[col]=df_general_check[col].astype('int8')
        
df_general_check.dtypes

MSSubClass           int64
MSZoning            string
Neighborhood        string
HouseStyle          string
OverallQual          int64
YearBuilt            int64
YearRemodAdd         int64
Exterior1st         string
Exterior2nd         string
MasVnrType          string
Foundation          string
BsmtQual            string
BsmtFinType1        string
GrLivArea            int64
TotRmsAbvGrd         int64
Fireplaces           int64
GarageType          string
GarageFinish        string
GarageCars           int64
GarageArea           int64
SalePrice            int64
OverallCond_bin       int8
FullBath_bin          int8
ExterQual_bin         int8
KitchenQual_bin       int8
MSZoning_cat          int8
Neighborhood_cat      int8
HouseStyle_cat        int8
Exterior1st_cat       int8
Exterior2nd_cat       int8
MasVnrType_cat        int8
Foundation_cat        int8
BsmtQual_cat          int8
BsmtFinType1_cat      int8
GarageType_cat        int8
GarageFinish_cat      int8
dtype: object

So let's look at descriptives for our continous variables:

In [41]:
rp.summary_cont(df_general_check[df_general_check.select_dtypes(include=["int64"]).columns])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,MSSubClass,1460.0,56.8973,42.3006,1.1071,54.7257,59.0689
1,OverallQual,1460.0,6.0993,1.3830,0.0362,6.0283,6.1703
2,YearBuilt,1460.0,1971.2678,30.2029,0.7904,1969.7173,1972.8183
3,YearRemodAdd,1460.0,1984.8658,20.6454,0.5403,1983.8059,1985.9256
4,GrLivArea,1460.0,1515.4637,525.4804,13.7525,1488.4870,1542.4404
5,TotRmsAbvGrd,1460.0,6.5178,1.6254,0.0425,6.4344,6.6013
6,Fireplaces,1460.0,0.6130,0.6447,0.0169,0.5799,0.6461
7,GarageCars,1460.0,1.7671,0.7473,0.0196,1.7288,1.8055
8,GarageArea,1460.0,472.9801,213.8048,5.5955,462.0040,483.9563
9,SalePrice,1460.0,180921.1959,79442.5029,2079.1053,176842.8410,184999.5507


We clearly see that the order of magnitude between the variables vary, therefore the data should be normalized. Normally we would do that only after splitting to folds (in order not to introduce data leakage), but this is just a general check, so let's do it right away

In [45]:
for col in df_general_check.select_dtypes(include=["int64"]).columns:
    name = "Z"+col
    df_general_check[name] = (df_general_check[col] - df_general_check[col].mean()) / df_general_check[col].std()
    
df_general_check.dtypes


MSSubClass            int64
MSZoning             string
Neighborhood         string
HouseStyle           string
OverallQual           int64
YearBuilt             int64
YearRemodAdd          int64
Exterior1st          string
Exterior2nd          string
MasVnrType           string
Foundation           string
BsmtQual             string
BsmtFinType1         string
GrLivArea             int64
TotRmsAbvGrd          int64
Fireplaces            int64
GarageType           string
GarageFinish         string
GarageCars            int64
GarageArea            int64
SalePrice             int64
OverallCond_bin        int8
FullBath_bin           int8
ExterQual_bin          int8
KitchenQual_bin        int8
MSZoning_cat           int8
Neighborhood_cat       int8
HouseStyle_cat         int8
Exterior1st_cat        int8
Exterior2nd_cat        int8
MasVnrType_cat         int8
Foundation_cat         int8
BsmtQual_cat           int8
BsmtFinType1_cat       int8
GarageType_cat         int8
GarageFinish_cat    

In [50]:
df_general_check[df_general_check.select_dtypes(include=["float64"]).columns].head()

,ZMSSubClass,ZOverallQual,ZYearBuilt,ZYearRemodAdd,ZGrLivArea,ZTotRmsAbvGrd,ZFireplaces,ZGarageCars,ZGarageArea,ZSalePrice
0,-0.872264,0.651256,1.116853,0.975241,0.415879,0.296662,-0.950901,0.311618,0.874722,0.995422
1,-0.872264,1.374324,0.322227,-0.187245,0.364498,-0.318574,0.600289,0.311618,0.177825,0.592615
2,-0.872264,0.651256,0.984415,0.829930,0.048596,-0.318574,0.600289,1.649742,1.730643,0.303097
3,0.073350,-0.071812,0.951306,0.733056,0.282668,0.296662,0.600289,0.311618,-0.060710,-0.107262
4,3.146594,-0.794879,-0.538617,-1.446605,-0.874369,-0.933810,-0.950901,0.311618,0.463132,-0.452166


And now let's see the descriptives for the categorical variables:

In [43]:
pd.set_option('display.max_rows', 107)
rp.summary_cat(df_general_check[df_general_check.select_dtypes(include=["int8"]).columns])

,Variable,Outcome,Count,Percent
0,OverallCond_bin,1,821,56.23
1,,0,639,43.77
2,FullBath_bin,1,768,52.60
3,,0,692,47.40
4,ExterQual_bin,1,920,63.01
5,,0,540,36.99
6,KitchenQual_bin,1,774,53.01
7,,0,686,46.99
8,MSZoning_cat,3,1151,78.84
9,,4,218,14.93


First of all - quite a few variables have multiple categories that are underrepresented in the data. Probably they should be transformed into joint larger ones in some way. As a rule o a thumb, let's aggregate all categories represented in below 10% of samples as one category "Other" and value 999.

In [80]:
for col in df_general_check.select_dtypes(include=["int8"]).columns:
    data=df_general_check[col].value_counts(normalize=True)
    df_temp=pd.DataFrame({'value':data.index, 'percentage':data.values})
    for value in df_temp.loc[df_temp['percentage']<0.1, 'value']:
        df_general_check[col] = np.where(df_general_check[col]==value, 999, df_general_check[col])
        
rp.summary_cat(df_general_check[df_general_check.select_dtypes(include=["int16"]).columns])

,Variable,Outcome,Count,Percent
0,MSZoning_cat,3,1151,78.84
1,,4,218,14.93
2,,999,91,6.23
3,Neighborhood_cat,999,1085,74.32
4,,12,225,15.41
5,,5,150,10.27
6,HouseStyle_cat,2,726,49.73
7,,5,445,30.48
8,,0,154,10.55
9,,999,135,9.25


As we can see - the neighberhood category will probably be dropped in the future

#### Overall regression model

Let's look at the left variables

In [84]:
df_general_check.columns

Index(['MSSubClass', 'MSZoning', 'Neighborhood', 'HouseStyle', 'OverallQual',
       'YearBuilt', 'YearRemodAdd', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'Foundation', 'BsmtQual', 'BsmtFinType1', 'GrLivArea', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageType', 'GarageFinish', 'GarageCars', 'GarageArea',
       'SalePrice', 'OverallCond_bin', 'FullBath_bin', 'ExterQual_bin',
       'KitchenQual_bin', 'MSZoning_cat', 'Neighborhood_cat', 'HouseStyle_cat',
       'Exterior1st_cat', 'Exterior2nd_cat', 'MasVnrType_cat',
       'Foundation_cat', 'BsmtQual_cat', 'BsmtFinType1_cat', 'GarageType_cat',
       'GarageFinish_cat', 'ZMSSubClass', 'ZOverallQual', 'ZYearBuilt',
       'ZYearRemodAdd', 'ZGrLivArea', 'ZTotRmsAbvGrd', 'ZFireplaces',
       'ZGarageCars', 'ZGarageArea', 'ZSalePrice'],
      dtype='object')

And let's model the regression with our variables

In [89]:
model = smf.ols("SalePrice ~ ZOverallQual + ZYearBuilt + ZYearRemodAdd + ZGrLivArea + ZTotRmsAbvGrd + ZFireplaces\
                + ZGarageCars + ZGarageArea\
                + C(OverallCond_bin) + C(FullBath_bin) + C(ExterQual_bin) + C(KitchenQual_bin) + C(MSZoning_cat)\
                + C(Neighborhood_cat) + C(Neighborhood_cat) + C(HouseStyle_cat) + C(Exterior1st_cat)\
                + C(Exterior2nd_cat) + C(MasVnrType_cat) + C(Foundation_cat) + C(BsmtQual_cat)\
                + C(BsmtFinType1_cat) + C(GarageType_cat) + C(GarageFinish_cat)", df_general_check).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.820
Model:                            OLS   Adj. R-squared:                  0.815
Method:                 Least Squares   F-statistic:                     154.0
Date:                Fri, 30 Sep 2022   Prob (F-statistic):               0.00
Time:                        17:31:59   Log-Likelihood:                -17291.
No. Observations:                1460   AIC:                         3.467e+04
Df Residuals:                    1417   BIC:                         3.490e+04
Df Model:                          42                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                   1.934e+05   7898.428     24.489      0.000    1.78e+05    2.09e+05
C(OverallCond_bin)[T.1]    -9339.0955   2466.818     -3.786      0.000   -1.42e+04   -4500.088
C(FullBath_bin)[T.1]       -1.732e+04   2554.684     -6.779      0.000   -2.23e+04   -1.23e+04
C(ExterQual_bin)[T.1]      -9559.5225   3267.634     -2.926      0.003    -1.6e+04   -3149.602
C(KitchenQual_bin)[T.1]    -5247.6497   2903.885     -1.807      0.071   -1.09e+04     448.726
C(MSZoning_cat)[T.4]       -1.546e+04   3058.252     -5.054      0.000   -2.15e+04   -9457.109
C(MSZoning_cat)[T.999]     -7100.4835   4155.328     -1.709      0.088   -1.53e+04    1050.772
C(Neighborhood_cat)[T.12]  -1574.0841   4410.521     -0.357      0.721   -1.02e+04    7077.768
C(Neighborhood_cat)[T.999]  6535.2797   3438.857      1.900      0.058    -210.518    1.33e+04
C(HouseStyle_cat)[T.2]      1.106e+04   3579.570      3.090      0.002    4037.527    1.81e+04
C(HouseStyle_cat)[T.5]     -4815.4244   3645.941     -1.321      0.187    -1.2e+04    2336.597
C(HouseStyle_cat)[T.999]    1352.2587   4421.891      0.306      0.760   -7321.898       1e+04
C(Exterior1st_cat)[T.8]     4439.0677   1.16e+04      0.382      0.702   -1.83e+04    2.72e+04
C(Exterior1st_cat)[T.12]    1.052e+04   1.05e+04      1.004      0.316      -1e+04    3.11e+04
C(Exterior1st_cat)[T.13]    4545.5479   7315.103      0.621      0.534   -9804.048    1.89e+04
C(Exterior1st_cat)[T.999]    1.11e+04   5858.627      1.895      0.058    -392.802    2.26e+04
C(Exterior2nd_cat)[T.8]     1176.5366   1.17e+04      0.100      0.920   -2.19e+04    2.42e+04
C(Exterior2nd_cat)[T.13]   -5512.6616   1.05e+04     -0.523      0.601   -2.62e+04    1.52e+04
C(Exterior2nd_cat)[T.14]    -250.7662   7396.799     -0.034      0.973   -1.48e+04    1.43e+04
C(Exterior2nd_cat)[T.999]  -5842.2220   5739.074     -1.018      0.309   -1.71e+04    5415.773
C(MasVnrType_cat)[T.2]      2171.0047   2326.208      0.933      0.351   -2392.177    6734.186
C(MasVnrType_cat)[T.999]    4372.5288   3523.445      1.241      0.215   -2539.200    1.13e+04
C(Foundation_cat)[T.1]       830.2032   4101.203      0.202      0.840   -7214.880    8875.286
C(Foundation_cat)[T.2]      4592.2074   4656.904      0.986      0.324   -4542.959    1.37e+04
C(Foundation_cat)[T.999]   -4703.7771   7226.044     -0.651      0.515   -1.89e+04    9471.116
C(BsmtQual_cat)[T.3]        3495.9789   3002.329      1.164      0.244   -2393.509    9385.467
C(BsmtQual_cat)[T.999]      3.451e+04   3452.480      9.996      0.000    2.77e+04    4.13e+04
C(BsmtFinType1_cat)[T.1]   -2327.4072   3738.376     -0.623      0.534   -9660.753    5005.939
C(BsmtFinType1_cat)[T.2]    6033.3113   3325.670      1.814      0.070    -490.455    1.26e+04
C(BsmtFinType1_cat)[T.5]   -1.438e+04   312

We can see that overall the model is significant and accounts for almost 80% of variance. The residuals are of normal distribution, which is good, but the condition number is quite high suggesting an existance of multicoliearity. Let's inspect that further

In [99]:
# This creates the design matrix used by StatsModels
x = patsy.dmatrix("ZOverallQual + ZYearBuilt + ZYearRemodAdd + ZGrLivArea + ZTotRmsAbvGrd + ZFireplaces\
                + ZGarageCars + ZGarageArea\
                + C(OverallCond_bin) + C(FullBath_bin) + C(ExterQual_bin) + C(KitchenQual_bin) + C(MSZoning_cat)\
                + C(Neighborhood_cat) + C(Neighborhood_cat) + C(HouseStyle_cat) + C(Exterior1st_cat)\
                + C(Exterior2nd_cat) + C(MasVnrType_cat) + C(Foundation_cat) + C(BsmtQual_cat)\
                + C(BsmtFinType1_cat) + C(GarageType_cat) + C(GarageFinish_cat)", data = df_general_check)


vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(x, i) for i in range(x.shape[1])]
vif["Features"] = x.design_info.column_names

vif

,VIF Factor,Features
0,77.987241,Intercept
1,1.872211,C(OverallCond_bin)[T.1]
2,2.034127,C(FullBath_bin)[T.1]
3,3.110892,C(ExterQual_bin)[T.1]
4,2.625792,C(KitchenQual_bin)[T.1]
5,1.485119,C(MSZoning_cat)[T.4]
6,1.261514,C(MSZoning_cat)[T.999]
7,3.170047,C(Neighborhood_cat)[T.12]
8,2.821796,C(Neighborhood_cat)[T.999]
9,4.004340,C(HouseStyle_cat)[T.2]


Three variables have FIV over 10, which indicates multicolinearity:
- C(Exterior1st_cat)[T.8]
- C(Exterior1st_cat)[T.12]
- C(Exterior2nd_cat)[T.13]

So, the high condition number is probably a byproduct of poor conditioning of the design matrix. Let's inspect that

In [100]:
x = np.asarray(x)

In [103]:
# Getting the singular values from SVD
_, sing_as, _ = np.linalg.svd(x)


# Calculating the condiction index
condition_index = []
for n in sing_as:
    ci = sing_as.max() / n
    condition_index.append(ci)

eigen_vals = (sing_as * sing_as).round(3)

pd.DataFrame(np.c_[eigen_vals, condition_index],
             columns = ["Eigenvalues", "Condition Index"]).round(1)


,Eigenvalues,Condition Index
0,8107.7,1.0
1,7121.6,1.1
2,2584.8,1.8
3,1417.8,2.4
4,1190.8,2.6
5,770.3,3.2
6,605.9,3.7
7,576.3,3.8
8,515.4,4.0
9,470.5,4.2


Condition Index is >30 for two values, hence given that the condition number is a method to identify collinearity, by definition of multicollinearity, there needs to be at least 2 independent variables that have some form of linear dependency if it exists; which is the current case.

Now to assess the eigenvalues, small eigenvalues indicate instability in the estimation of the regression coeffiecients and multiple small eigenvalues indicates intercorrelation.

#### SUMMARY

So, to summarize our general check:
multiple variables should have a grouping category "Other" introduced
variables: Exterior1st_cat, Exterior2nd_cat, MSSubClass and Neighborhood_cat should be left out of the final model